# 第二章：CUDA简介

2006年11月，NVIDIA®推出了CUDA®(统一计算设备架构)，这是一个通用并行计算平台和编程模型，它利用NVIDIA GPU中的并行计算引擎来比CPU更高效地解决许多复杂的计算问题。

如果使用高级库接口，许多Python开发人员可以跳过本章。但如果您需要编写自己的内核函数或将算法高效地映射到GPU架构中，本章介绍的概念将会派上用场。

## CUDA架构

在科学代码中在GPU上执行函数(也称为内核)通常需要三个主要步骤：
1. 将输入数据从CPU内存复制到GPU内存
2. 在GPU上加载并执行GPU内核
3. 将结果从GPU内存复制到CPU内存

正如上一章所述，CPU和GPU的架构是不同的，需要不同的编程范式。


## CUDA编程模型

CUDA在涉及CPU和GPU的异构计算模型上运行。

### 内核

CUDA内核是在GPU上执行的全局函数。应用程序的并行部分由K个不同的CUDA线程并行执行K次，而不是像CPU上的常规顺序函数那样只执行一次。

### 线程和线程束

在CUDA中，内核通过线程来执行。线程是表示内核执行的抽象实体。这代表了CUDA中最小的执行单位，并拥有唯一的线程ID。

每32个连续的线程称为一个线程束。线程束是SM中的主要执行单位。一旦线程块被分配给SM，它将被进一步划分为一组线程束来执行。每个线程块始终有固定数量的线程束，GPU上的操作是以线程束为单位执行的。

### 块

一组线程称为CUDA块。这组线程可以通过共享内存进行协作，并同步它们的内存访问和执行。CUDA块被组织成网格。内核作为线程块网格来执行。块数组执行相同的内核，将并行执行扩展到所有线程。

每个CUDA块由一个流式多处理器(SM)执行，不能迁移到GPU中的其他SM(除了在抢占、调试或CUDA动态并行性期间)。一个SM可以根据CUDA块需要的资源运行多个并发CUDA块。每个内核在一个设备上执行，CUDA支持在一个设备上同时运行多个内核。

_GPU内核执行_

![KernelExecution](images/chapter-02/gpu-kernel-exec.png)

这些组件的层次结构不仅有助于线程执行的管理，还反映了支持CUDA的GPU的物理架构。线程在多个流式多处理器(SM)上并发执行。


## 内存管理

理解CUDA内存层次结构对于优化CUDA程序的性能至关重要。GPU具有几种不同类型的内存，每种类型都有自己的作用域和缓存。以有效利用共享内存的方式使用内存类型可以减少延迟并提高吞吐量。

主要类型包括：
- 全局内存 - 所有线程都可以访问。
- 共享内存/L1缓存 - 在同一块内的线程之间共享。
- 局部内存 - 对每个线程私有，但是全局内存的一部分。
- 寄存器 - 对每个线程私有。

![GPUMemory](images/chapter-02/gpu-memory.png)

还有其他类型的内存需要注意，但可能超出使用高级库构建加速Python应用程序的范围。
- 常量内存 - 所有线程都可以访问的缓存只读内存。在NVIDIA GPU的情况下，共享内存、L1缓存和常量内存缓存都在流式多处理器块内。因此它们比L2缓存和GPU RAM更快。
- 纹理内存 - 为所有线程优化的只读内存，用于过滤、插值方法和随机访问。

内存类型会影响性能。例如，全局内存容量大但可能增加延迟。局部内存对每个线程是私有的，但是全局内存的一部分，所以访问时间也可能较长。共享内存更快，但大小有限。

为了获得最佳性能，数据应该位于最接近处理位置的内存类型中，并且最匹配其访问模式。经常访问或在线程之间共享的数据应尽可能移动到共享内存或存储在寄存器中。这将减少从全局内存获取数据相关的开销。

_内存层次结构_

![GPUMemoryHierarchy](images/chapter-02/gpu-memory-hierarchy.png)

该架构创建了一个托管内存池，从这个内存池中的每个分配都可以在主机和设备上使用相同的地址或指针访问。底层系统将数据迁移到主机和设备。

在GPU加速应用程序中实现最佳性能的关键是能够在CPU和GPU之间高效地传输数据。CPU和GPU之间的数据传输涉及通过PCIe总线移动数据，这可能会限制访问CPU和GPU上本地内存的速度。


## 以Python方式前进

这些内存分配类型在本章中作为参考，但在许多高级CUDA Python包中，内核执行、线程执行和内存管理的细节通常是隐藏的。大量生成的样板代码和智能默认值导致性能良好的代码，而无需调用低级CUDA功能。不熟悉C++的Python开发人员也会欣赏API的Python风格特性。

普遍的共识是，开发人员应该使用可用的最高级库来开发他们的应用程序，然后在必要时探索低级选项。大多数高级包都提供了访问低级功能的接口，所以如果高级库调用无法提供您所需的性能，您可以使用这个选项。


## 资源

CUDA C++编程指南：https://docs.nvidia.com/cuda/cuda-c-programming-guide/ 

CUDA工具包：https://developer.nvidia.com/cuda-toolkit 

NVIDIA CUDA开发者论坛：https://forums.developer.nvidia.com/c/accelerated-computing/cuda/206 